In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from scipy.stats import mode
import json

Blood Transfusion Service Center Data Set is used, which can be downloaded from here - 
https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center

The Attribute information of the dataset is given below:

**Recency**                - Months since previous donation

**Frequency**              - Number of donations

**Monetary**               - total donations

**Time**                   - Months since first donation

**Donates Blood in March** - Yes(1)/No(0)

In [2]:
df = pd.read_csv(r".\dataset\transfusion.data",sep=',')

df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [3]:
df = df.rename(columns={'Recency (months)':'Recency','Frequency (times)':'Frequency','Monetary (c.c. blood)':'Monetary',\
                   'Time (months)':'Time','whether he/she donated blood in March 2007':'Blood Donated'})

data = np.array(df)

### Split Data

In [4]:
def split_data(value,column_index,data):
    left = data[data[:,column_index] <= value]
    right = data[data[:,column_index] > value]
    
    return left,right

Lets test the above method by giving some column index and some random value from that column

The Column Index of the dataset is given below(within paranthesis):

**Recency**                - {0}

**Frequency**              - {1}

**Monetary**               - {2}

**Time**                   - {3}

**Donates Blood in March** - {4}

In [5]:
column_index = np.random.choice(data.shape[-1]-1)
print("The column index on which the split occurs:",column_index)
value = np.random.choice(data[:,column_index])
print("The Split Value:",value)
nodes = split_data(value,column_index,data)

The column index on which the split occurs: 1
The Split Value: 1


### Cost Function(Gini Index Minimization)

In [6]:
def Gini_Index(groups,classes):
    total_samples = sum(len(group) for group in groups)
    gini_index = 0
    for group in groups:
        size = len(group)
        score = 0
        if size == 0:
            continue
        for class_label in classes:
            p = sum(group[:,-1]==class_label)/size
            score += p*p
        
        gini_index += (1 - score)*(size/total_samples)
    
    return gini_index

In [7]:
classes = [0,1]
gini_index = Gini_Index(nodes,classes)
print("The gini index for the split on Column Index:",column_index,"and split value of:",value,"is:",gini_index)

The gini index for the split on Column Index: 1 and split value of: 1 is: 0.356033395785027


### Fetching Optimal split value

In [8]:
def getOptimalSplit_value(data):
    classes = np.unique(data[:,-1])
    best_index,best_gini,best_splitval,best_group = 9999,9999,9999,None
    for index in range(data.shape[1] - 1):
        for row in data:
            groups = split_data(row[index],index,data)
            gini_index = Gini_Index(groups,classes)
            if gini_index < best_gini:
                best_index,best_gini,best_splitval,best_group = index,gini_index,row[index],groups
    
    return {
        "index":best_index,
        "gini":best_gini,
        "value":best_splitval,
        "group":best_group
           }

In [9]:
split = getOptimalSplit_value(data)

print("Best Split Occurs for Column:",split['index'],"for Value:",split['value']
     ,"with gini index:",split['gini'])

Best Split Occurs for Column: 0 for Value: 6 with gini index: 0.3273963559783137


**Note:** The above optimal split value computed will act as a root node in constructing the tree further 

## Build the tree

In [10]:
# This can be used to return the most probable value that the node can return based what class type data does it have most
def terminal_nodes(groups):
    return int(mode(groups[:,-1])[0])

In [11]:
# Build the tree recursively 
def Recursive_split(node,depth,maxdepth,min_size):
    left,right = node['group']
    
    del(node['group'])
    
    if (left.size == 0) or (right.size == 0):
        array = np.array(list(left)+list(right))
        node['left'] = node['right'] = terminal_nodes(array)
        return
    
    if depth>=maxdepth:
        node['left'],node['right'] = terminal_nodes(left),terminal_nodes(right)
        return
    
    if len(left)<=min_size:
        node['left'] = terminal_nodes(left)
        
    else:
        node['left'] = getOptimalSplit_value(left)
        Recursive_split(node['left'],depth+1,maxdepth,min_size)
    
    if len(right)<=min_size:
        node['right'] = terminal_nodes(right)
        
    else:
        node['right'] = getOptimalSplit_value(right)
        Recursive_split(node['right'],depth+1,maxdepth,min_size)

In [12]:
def build_tree(data,max_depth,min_size):
    root = getOptimalSplit_value(data)
    Recursive_split(root,1,max_depth,min_size)
    return root

In [13]:
tree = build_tree(data,1,1)
print(tree)

{'index': 0, 'gini': 0.3273963559783137, 'value': 6, 'left': 0, 'right': 0}


In [14]:
tree =build_tree(data,2,1)
print(tree)
tree = build_tree(data,3,1)
print(tree)

{'index': 0, 'gini': 0.3273963559783137, 'value': 6, 'left': {'index': 1, 'gini': 0.44148203603730607, 'value': 4, 'left': 0, 'right': 0}, 'right': {'index': 0, 'gini': 0.18962919069309306, 'value': 14, 'left': 0, 'right': 0}}
{'index': 0, 'gini': 0.3273963559783137, 'value': 6, 'left': {'index': 1, 'gini': 0.44148203603730607, 'value': 4, 'left': {'index': 3, 'gini': 0.37521543114763445, 'value': 16, 'left': 0, 'right': 0}, 'right': {'index': 3, 'gini': 0.4505597049791006, 'value': 49, 'left': 1, 'right': 0}}, 'right': {'index': 0, 'gini': 0.18962919069309306, 'value': 14, 'left': {'index': 3, 'gini': 0.23717168406760603, 'value': 15, 'left': 0, 'right': 0}, 'right': {'index': 1, 'gini': 0.12702100662625124, 'value': 4, 'left': 0, 'right': 0}}}


In [15]:
tree = build_tree(data,10,1)
print(tree)

{'index': 0, 'gini': 0.3273963559783137, 'value': 6, 'left': {'index': 1, 'gini': 0.44148203603730607, 'value': 4, 'left': {'index': 3, 'gini': 0.37521543114763445, 'value': 16, 'left': {'index': 3, 'gini': 0.39378156565656564, 'value': 2, 'left': {'index': 1, 'gini': 0.2727272727272726, 'value': 1, 'left': {'index': 0, 'gini': 0.2975206611570247, 'value': 2, 'left': 0, 'right': 0}, 'right': {'index': 0, 'gini': 0.0, 'value': 2, 'left': 0, 'right': 0}}, 'right': {'index': 3, 'gini': 0.4276315789473684, 'value': 3, 'left': 1, 'right': {'index': 1, 'gini': 0.42325814536340856, 'value': 1, 'left': {'index': 0, 'gini': 0.3526530612244898, 'value': 4, 'left': 0, 'right': 0}, 'right': {'index': 3, 'gini': 0.4505287896592245, 'value': 11, 'left': {'index': 0, 'gini': 0.46949806949806955, 'value': 0, 'left': {'index': 0, 'gini': 0.0, 'value': 0, 'left': 0, 'right': 0}, 'right': {'index': 0, 'gini': 0.4789915966386555, 'value': 4, 'left': {'index': 1, 'gini': 0.4842826019296608, 'value': 2, 'le

The Tree variable contains the structure of the tree on the data. Lets try to predict on test data. 

## Prediction

In [16]:
def Predict(node,test_row):
    if test_row[node['index']] < node['value']:
        if isinstance(node['left'],dict):
            return Predict(node['left'],test_row)
        else:
            return node['left']
    else:
        if isinstance(node['right'],dict):
            return Predict(node['right'],test_row)
        else:
            return node['right']

In [17]:
def Get_Prediction(tree,data):
    Predictions = []
    for row in data:
        Prediction = Predict(tree,row)
        Predictions.append(Prediction)
    return np.array(Predictions)

## Main Method

Lets split the data into 70:30 % between train and test data.

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[:,0:-1],data[:,-1],test_size=0.3,random_state=30)

In [19]:
max_depth = 5
min_size = 7
train_data = np.append(X_train,y_train.reshape(y_train.shape[0],1),axis=1)
test_data = np.append(X_test,y_test.reshape(y_test.shape[0],1),axis=1)

In [20]:
#Build the tree using the train dataset
tree = build_tree(train_data,max_depth,min_size)

In [21]:
# Get Predictions on the train data
Predictions_train = Get_Prediction(tree,train_data)

In [22]:
# Accuracy on the train data:
print("The Accuracy on the training set is:",np.mean(Predictions_train == y_train)*100)

The Accuracy on the training set is: 78.39388145315488


In [23]:
# Get Predictions on the test data
Predictions_test = Get_Prediction(tree,test_data)

# Accuracy on the test data:
print("The Accuracy on the testing set is:",np.mean(Predictions_test == y_test)*100)

The Accuracy on the testing set is: 76.44444444444444


## Finding Optimal Dept and Size for the Tree (Hyperparameter Tuning)

In [24]:
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.7*len(df)), int(.85*len(df))])

In [25]:
train, validate, test = np.array(train),np.array(validate),np.array(test)

I am trying with couple of values of max_dept and min_size on the cross validation set to find the optimal values for the same(The one combination which returns best accuracy)

**Note:** We can try to find the optimal values for max_depth and min_size separately. But i have opted to find them combined.

In [29]:
max_depth = [2,3,4,5,6,7,8]
min_size = [5,6,7,8,9,10]

In [30]:
def find_Optimal_dept_size(max_depth,min_size,train,validate):
    
    best_depth,best_size,best_accuracy = 999,999,-1
    
    y_val = validate[:,-1]
    
    for depth in max_depth:
        
        for size in min_size:
            
            # Build the tree
            tree = build_tree(train,depth,size)
            
            #Find Predictions on the validation set
            Predictions = Get_Prediction(tree,validate)
            
            #find the accuracy
            accuracy = np.mean(Predictions == y_val)*100
            
            # Print the accuracy obtained for each combination of depth and size
            print("For depth =",depth,"and size =",size,"accuracy is:",accuracy)
            
            if accuracy > best_accuracy:
                best_depth,best_size,best_accuracy = depth,size,accuracy
        
    return({
        "Accuracy":best_accuracy,
        "Depth":best_depth,
        "Size":best_size
    })
        
     

In [31]:
best_values = find_Optimal_dept_size(max_depth,min_size,train,validate)

For depth = 2 and size = 5 accuracy is: 80.35714285714286
For depth = 2 and size = 6 accuracy is: 80.35714285714286
For depth = 2 and size = 7 accuracy is: 80.35714285714286
For depth = 2 and size = 8 accuracy is: 80.35714285714286
For depth = 2 and size = 9 accuracy is: 80.35714285714286
For depth = 2 and size = 10 accuracy is: 80.35714285714286
For depth = 3 and size = 5 accuracy is: 75.0
For depth = 3 and size = 6 accuracy is: 75.0
For depth = 3 and size = 7 accuracy is: 75.0
For depth = 3 and size = 8 accuracy is: 75.0
For depth = 3 and size = 9 accuracy is: 75.0
For depth = 3 and size = 10 accuracy is: 75.0
For depth = 4 and size = 5 accuracy is: 75.89285714285714
For depth = 4 and size = 6 accuracy is: 75.89285714285714
For depth = 4 and size = 7 accuracy is: 75.89285714285714
For depth = 4 and size = 8 accuracy is: 75.89285714285714
For depth = 4 and size = 9 accuracy is: 75.89285714285714
For depth = 4 and size = 10 accuracy is: 75.89285714285714
For depth = 5 and size = 5 accu

In [32]:
print("The best optimal values for depth is:",best_values['Depth'],"and size is:",best_values['Size'],\
      "for accuracy of:",best_values['Accuracy'])

The best optimal values for depth is: 2 and size is: 5 for accuracy of: 80.35714285714286


In [33]:
#Lets build the tree with above depth and size 
y_test = test[:,-1]
tree = build_tree(train,best_values['Depth'],best_values['Size'])

#Find Predictions on the test set
Predictions = Get_Prediction(tree,test)

#find the accuracy
accuracy = np.mean(Predictions == y_test)*100


In [34]:
print("The accuracy on the test set:",accuracy)

The accuracy on the test set: 80.53097345132744
